In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import TextVectorization

# 1) Define as constantes
MAXLEN_DOC = 100
MAXLEN_SUM = 30
VOCAB_IN  = 10_000
VOCAB_OUT = 6_000
BATCH     = 16_384
AUTOTUNE  = tf.data.AUTOTUNE


2025-07-30 13:24:59.854801: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753892700.507225    6881 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753892700.673501    6881 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753892702.342645    6881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753892702.342674    6881 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753892702.342677    6881 computation_placer.cc:177] computation placer alr

In [2]:
# 2) Carrega e pré‑processa textualmente com TFDS
ds = tfds.load('Gigaword', split='train', shuffle_files=True, data_dir='../data')
 
for example in ds.take(1): # Take one example to inspect
    print(example)

I0000 00:00:1753892725.383056    6881 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1814 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-07-30 13:25:25.726559: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


{'document': <tf.Tensor: shape=(), dtype=string, numpy=b'russia and its pro-west neighbor georgia engaged in fierce fighting saturday in the disputed region of south ossetia , reports said , as the international community scrambled to prevent an all-out war .'>, 'summary': <tf.Tensor: shape=(), dtype=string, numpy=b'fierce fighting reported in georgia after russian troop surge'>}


2025-07-30 13:25:28.292124: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [3]:
ds = ds.map(lambda ex: {
    'document': tf.strings.regex_replace(ex['document'], r"\s+", " "),
    'summary' : tf.strings.regex_replace(ex['summary'],  r"\s+", " ")
})

for example in ds.take(1):
    print(example)

{'document': <tf.Tensor: shape=(), dtype=string, numpy=b"firemen rescued ## people from a high - rise building fire on thursday afternoon in northeast china 's heilongjiang province .">, 'summary': <tf.Tensor: shape=(), dtype=string, numpy=b'high-rise in ne china catches fire ## rescued'>}


2025-07-30 13:25:30.291058: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# 3) Cria camadas TextVectorization
tv_doc = TextVectorization(
    max_tokens=VOCAB_IN,
    output_sequence_length=MAXLEN_DOC,
    standardize='lower_and_strip_punctuation'
)
tv_sum = TextVectorization(
    max_tokens=VOCAB_OUT,
    output_sequence_length=MAXLEN_SUM,
    standardize='lower_and_strip_punctuation'
)

In [5]:
# 4) “Adapta” (treina) o vocabulário nos textos brutos
docs_ds = ds.map(lambda ex: ex['document'])
sums_ds = ds.map(lambda ex: ex['summary'])
tv_doc.adapt(docs_ds.batch(1024))
tv_sum.adapt(sums_ds.batch(1024))

2025-07-30 13:27:01.490027: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
# 5) Pipeline de vetorização puramente em TF
def vectorize_tf(ex):
    return {
        'document': tv_doc(ex['document']),
        'summary' : tv_sum(ex['summary'])
    }

In [7]:
vectorized = (
    ds
    .map(vectorize_tf, num_parallel_calls=AUTOTUNE)
    .cache()                             # materializa uma vez em disco
    .batch(BATCH)
    .prefetch(AUTOTUNE)
)

In [10]:
for example in vectorized.take(1):
    print(example['document'].shape)
    print(example['summary'].shape)

(16384, 100)
(16384, 30)


2025-07-28 21:31:27.427389: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [11]:
# 6) Salva em disco de forma nativa (muito rápido, C/C++)
tf.data.Dataset.save(
    vectorized, 
    "vectorized_gigaword_ds"
)

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


In [13]:
import numpy as np

# Salvar vocabulário
vocab_doc = tv_doc.get_vocabulary()
np.save("/home/olavo-dalberto/models/vocab_doc.npy", vocab_doc)

vocab_sum = tv_sum.get_vocabulary()
np.save("/home/olavo-dalberto/models/vocab_sum.npy", vocab_sum)

# Carregar vocabulário
vocab_doc_loaded = np.load("/home/olavo-dalberto/models/vocab_doc.npy", allow_pickle=True)
vocab_sum_loaded = np.load("/home/olavo-dalberto/models/vocab_sum.npy", allow_pickle=True)

In [ ]:
print(vocab_doc_loaded)

In [14]:
# Salvar modelo Keras com TextVectorization
model_doc = tf.keras.Sequential([tv_doc])  # seu pipeline
model_doc.save("/home/olavo-dalberto/models/tv_doc_model.keras")

model_sum = tf.keras.Sequential([tv_sum])  # seu pipeline
model_sum.save("/home/olavo-dalberto/models/tv_sum_model.keras")

# Carregar depois
loaded_model = tf.keras.models.load_model("/home/olavo-dalberto/models/tv_doc_model.keras")

/mnt/dados/miniconda3/envs/ia/lib/python3.10/site-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


In [15]:
loaded_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)